In [1]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('election_system.db')
cursor = conn.cursor()

# Create tables if they do not exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS admins (
        id INTEGER PRIMARY KEY,
        password TEXT NOT NULL
    )
''')

# Insert admin credentials if not already present
def insert_admin(admin_id, password):
    cursor.execute('''
        INSERT OR IGNORE INTO admins (id, password)
        VALUES (?, ?)
    ''', (admin_id, password))
    conn.commit()

# Example: Insert admin with ID 1 and password '12345'
insert_admin(1, '12345')

print("Admin credentials have been set up.")

conn.close()


Admin credentials have been set up.


In [3]:
import sqlite3
from datetime import datetime, timedelta
import random
import string

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('election_system.db')
cursor = conn.cursor()

# Create tables if they do not exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS admins (
        id INTEGER PRIMARY KEY,
        password TEXT NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS constituencies (
        name TEXT PRIMARY KEY,
        commencement_date TEXT,
        end_date TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS voters (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        full_name TEXT NOT NULL,
        constituency TEXT,
        dob TEXT,
        password TEXT NOT NULL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS candidates (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        constituency TEXT NOT NULL,
        FOREIGN KEY (constituency) REFERENCES constituencies(name)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS votes (
        voter_id INTEGER,
        candidate_id INTEGER,
        FOREIGN KEY (voter_id) REFERENCES voters(id),
        FOREIGN KEY (candidate_id) REFERENCES candidates(id),
        PRIMARY KEY (voter_id, candidate_id)
    )
''')


# Check admin credentials
def validate_admin(admin_id, password):
    cursor.execute('''
        SELECT * FROM admins WHERE id = ? AND password = ?
    ''', (admin_id, password))
    return cursor.fetchone()

# Add a new constituency
def add_constituency(name, commencement_date, end_date):
    cursor.execute('''
        INSERT INTO constituencies (name, commencement_date, end_date)
        VALUES (?, ?, ?)
    ''', (name, commencement_date, end_date))
    conn.commit()

# Check if constituency already exists
def constituency_exists(name):
    cursor.execute('''
        SELECT * FROM constituencies WHERE name = ?
    ''', (name,))
    return cursor.fetchone() is not None

# Add a new voter
def add_voter(full_name, constituency, dob):
    password = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
    cursor.execute('''
        INSERT INTO voters (full_name, constituency, dob, password)
        VALUES (?, ?, ?, ?)
    ''', (full_name, constituency, dob, password))
    conn.commit()
    return password

# Validate voter credentials
def validate_voter(voter_id, password):
    cursor.execute('''
        SELECT * FROM voters WHERE id = ? AND password = ?
    ''', (voter_id, password))
    return cursor.fetchone()

# Get constituency details
def get_constituency_details(name):
    cursor.execute('''
        SELECT * FROM constituencies WHERE name = ?
    ''', (name,))
    return cursor.fetchone()

# Function to add a candidate
def add_candidate(name, constituency):
    cursor.execute('INSERT INTO candidates (name, constituency) VALUES (?, ?)', (name, constituency))
    conn.commit()
    print(f"Candidate {name} added to {constituency}.")

# Function to add candidate via admin action
def add_candidate_action():
    name = input("Enter Candidate Name: ")
    constituency = input("Enter Constituency Name: ")

    if not constituency_exists(constituency):
        print("Constituency does not exist!")
        return
    
    add_candidate(name, constituency)

# Main menu
def main_menu():
    while True:
        print("Maharashtra Legislative Assembly Election 2022 Conducted by State Election Commission")
        print("1 if you're an ADMIN")
        print("2 if you're a VOTER")
        print("3 to EXIT")
        choice = input("Enter choice: ")

        if choice == '1':
            admin_login()
        elif choice == '2':
            voter_login()
        elif choice == '3':
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please try again.")

# Admin login
def admin_login():
    print("Welcome! You have 3 login attempts.")
    attempts = 3
    while attempts > 0:
        admin_id = input("Enter Admin ID: ")
        password = input("Enter password: ")

        if validate_admin(admin_id, password):
            print(f"Welcome Admin {admin_id}")
            admin_actions()
            return
        else:
            attempts -= 1
            print(f"Invalid credentials. You have {attempts} attempt(s) left.")
    print("No attempts left. Exiting...")
    return

# Admin actions
def admin_actions():
    while True:
        print("Press:")
        print("1 to Conduct Election")
        print("2 to Add Voter")
        print("3 to Update Voter")
        print("4 to Add Candidate")
        print("5 to LOG OUT")
        choice = input("Enter choice: ")

        if choice == '1':
            conduct_election()
        elif choice == '2':
            add_voter_action()
        elif choice == '3':
            update_voter()
        elif choice == '4':
            add_candidate_action()
        elif choice == '5':
            print("Logging out...")
            return
        else:
            print("Invalid choice. Please try again.")

# Conduct election
def conduct_election():
    while True:
        name = input("Enter Constituency Name: ")
        if constituency_exists(name):
            print("Election details already present for this Constituency!")
        else:
            print("Constituency Added!")
            month = int(input("Enter Month: "))
            day = int(input("Enter Day: "))
            hour = int(input("Enter Hour: "))
            minute = int(input("Enter Minutes: "))
            
            try:
                # Validate future date
                commencement_date = datetime(2024, month, day, hour, minute)
                if commencement_date <= datetime.now():
                    print("Please set a date in the future!")
                    continue
                
                duration = int(input("Enter Election Duration in Days: "))
                end_date = commencement_date + timedelta(days=duration)
                
                add_constituency(name, commencement_date.isoformat(), end_date.isoformat())
                print(f"Election Start time is {commencement_date.strftime('%a %b %d %H:%M:%S %Y')}")
                print(f"Election End time is {end_date.strftime('%a %b %d %H:%M:%S %Y')}")
                print(f"Election Details:\nConstituency: {name}\nFrom: {commencement_date.strftime('%Y/%m/%d/%H:%M:%S')}\nTo: {end_date.strftime('%Y/%m/%d/%H:%M:%S')}")
                return
            except ValueError as e:
                print(f"Error: {e}. Please enter valid date and time.")

# Add voter action
def add_voter_action():
    full_name = input("Full Name: ")
    constituency = input("Please Choose Constituency: ")
    dob = input("Please Enter Date of Birth in YYYY/MM/DD format: ")

    if constituency_exists(constituency):
        password = add_voter(full_name, constituency, dob)
        print(f"Voter details added:\nName: {full_name}\nConstituency: {constituency}\nDate of Birth: {dob}\nPassword: {password}")
    else:
        print("Constituency does not exist!")

# Update voter action
def update_voter():
    voter_id = input("Enter Voter ID to update: ")
    
    # Check if voter exists
    cursor.execute('SELECT * FROM voters WHERE id = ?', (voter_id,))
    voter = cursor.fetchone()
    if not voter:
        print("Voter ID does not exist!")
        return
    
    print("Update:\n1 Name\n2 Constituency\n3 Date of Birth\n4 Return to Menu")
    choice = input("Enter choice: ")

    if choice == '1':
        new_name = input("Enter new full name: ")
        cursor.execute('UPDATE voters SET full_name = ? WHERE id = ?', (new_name, voter_id))
    elif choice == '2':
        new_constituency = input("Enter new constituency: ")
        if constituency_exists(new_constituency):
            cursor.execute('UPDATE voters SET constituency = ? WHERE id = ?', (new_constituency, voter_id))
        else:
            print("Constituency does not exist!")
            return
    elif choice == '3':
        new_dob = input("Enter new Date of Birth in YYYY/MM/DD format: ")
        cursor.execute('UPDATE voters SET dob = ? WHERE id = ?', (new_dob, voter_id))
    elif choice == '4':
        return
    else:
        print("Invalid choice.")
        return
    
    conn.commit()
    print("Voter details updated!")

# Voter login
def voter_login():
    print("Welcome! You have 3 login attempts.")
    attempts = 3
    while attempts > 0:
        voter_id = input("Please Enter Voter ID: ")
        password = input("Please Enter Password: ")

        if validate_voter(voter_id, password):
            print(f"Welcome Voter {voter_id}!")
            voter_actions(voter_id)
            return
        else:
            attempts -= 1
            print(f"Invalid credentials. You have {attempts} attempt(s) left.")
    print("No attempts left. Exiting...")
    return

# Voter actions
def voter_actions(voter_id):
    while True:
        print("Press:")
        print("1 to VOTE")
        print("2 to CHECK RESULT")
        print("3 to CHECK LIVE POLL")
        print("4 to LOG OUT")
        choice = input("Enter choice: ")

        if choice == '1':
            vote(voter_id)
        elif choice == '2':
            check_result()
        elif choice == '3':
            check_live_poll()
        elif choice == '4':
            print("Logging out...")
            return
        else:
            print("Invalid choice. Please try again.")

# Voting functionality
def vote(voter_id):
    cursor.execute('SELECT constituency FROM voters WHERE id = ?', (voter_id,))
    voter = cursor.fetchone()
    if not voter:
        print("Voter not found.")
        return

    constituency_name = voter[0]
    cursor.execute('SELECT id, name FROM candidates WHERE constituency = ?', (constituency_name,))
    candidates = cursor.fetchall()
    
    if not candidates:
        print("No candidates available for this constituency.")
        return

    print("Candidates:")
    for candidate in candidates:
        print(f"{candidate[0]}: {candidate[1]}")

    candidate_id = input("Enter the ID of the candidate you want to vote for: ")

    # Check if the voter has already voted
    cursor.execute('SELECT * FROM votes WHERE voter_id = ? AND candidate_id = ?', (voter_id, candidate_id))
    if cursor.fetchone():
        print("You have already voted for this candidate.")
        return

    cursor.execute('INSERT INTO votes (voter_id, candidate_id) VALUES (?, ?)', (voter_id, candidate_id))
    conn.commit()
    print("Your vote has been recorded.")


    

# Check election result
def check_result():
    constituency = input("Enter Constituency Name: ")
    cursor.execute('SELECT id, name FROM candidates WHERE constituency = ?', (constituency,))
    candidates = cursor.fetchall()
    
    if not candidates:
        print("No candidates found for this constituency.")
        return

    print("Results:")
    for candidate in candidates:
        candidate_id = candidate[0]
        cursor.execute('SELECT COUNT(*) FROM votes WHERE candidate_id = ?', (candidate_id,))
        vote_count = cursor.fetchone()[0]
        print(f"Candidate: {candidate[1]}, Votes: {vote_count}")


# Check live poll status
def check_live_poll():
    constituency = input("Enter Constituency Name: ")
    cursor.execute('SELECT commencement_date, end_date FROM constituencies WHERE name = ?', (constituency,))
    election = cursor.fetchone()
    
    if not election:
        print("No election details found for this constituency.")
        return

    commencement_date, end_date = election
    commencement_date = datetime.fromisoformat(commencement_date)
    end_date = datetime.fromisoformat(end_date)
    
    if datetime.now() < commencement_date:
        print(f"Poll has not started yet. It will start on {commencement_date.strftime('%a %b %d %H:%M:%S %Y')}.")
    elif datetime.now() > end_date:
        print(f"Poll has ended. It ended on {end_date.strftime('%a %b %d %H:%M:%S %Y')}.")
    else:
        print("Poll is currently live.")
        cursor.execute('SELECT id, name FROM candidates WHERE constituency = ?', (constituency,))
        candidates = cursor.fetchall()
        
        if not candidates:
            print("No candidates available for this constituency.")
            return

        print("Current Candidates:")
        for candidate in candidates:
            candidate_id = candidate[0]
            cursor.execute('SELECT COUNT(*) FROM votes WHERE candidate_id = ?', (candidate_id,))
            vote_count = cursor.fetchone()[0]
            print(f"Candidate: {candidate[1]}, Votes: {vote_count}")


if __name__ == "__main__":
    main_menu()
    conn.close()


Maharashtra Legislative Assembly Election 2022 Conducted by State Election Commission
1 if you're an ADMIN
2 if you're a VOTER
3 to EXIT


Enter choice:  1


Welcome! You have 3 login attempts.


Enter Admin ID:  12345
Enter password:  1


Invalid credentials. You have 2 attempt(s) left.


...

Poll has not started yet. It will start on Thu Oct 10 02:30:00 2024.
Press:
1 to VOTE
2 to CHECK RESULT
3 to CHECK LIVE POLL
4 to LOG OUT


Enter choice:  4


Logging out...
Maharashtra Legislative Assembly Election 2022 Conducted by State Election Commission
1 if you're an ADMIN
2 if you're a VOTER
3 to EXIT


Enter choice:  3


Exiting...
